In [3]:
!pip install torch torch_geometric

In [4]:
import torch
from torch_geometric.datasets import CoraFull
from torch_geometric.data import DataLoader

In [5]:
from torch_geometric.data import DataLoader, NeighborSampler
from torch_geometric.data import ClusterLoader
dataset = CoraFull(root="cora")
data = dataset[0]
# loader = ClusterLoader(dataset, batch_size=32, shuffle=True)
# loader = DataLoader(dataset, batch_size=32, shuffle=True)
data.y[0]
# for batch in loader:
#   print(len(batch.x))

tensor(0)

In [6]:
y = data.y[0]  # Single label
print(y)
y = y.unsqueeze(0)
print(y)

tensor(0)
tensor([0])


In [7]:
class MeanAggregator(torch.nn.Module):
    def __init__(self, in_features, out_features, neigh_input_dim):
        super(MeanAggregator, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.neigh_input_dim = neigh_input_dim

        self.w = torch.nn.Parameter(torch.empty(neigh_input_dim + in_features, out_features))
        torch.nn.init.xavier_uniform_(self.w)

    def forward(self, x, sampled_neighbors):
        neighbors_message = torch.mean(sampled_neighbors, dim=0)
        message = torch.concat((x, neighbors_message), 0)
        weighted_message = torch.matmul(message, self.w)
        return weighted_message

In [11]:
class GraphSAGE(torch.nn.Module):
  def __init__(self):
    super(GraphSAGE, self).__init__()
    self.aggr1 = MeanAggregator(8710, 5000, 8710)
    self.aggr2 = MeanAggregator(5000, 1000, 8710)
    self.aggr3 = MeanAggregator(1000, 70, 8710)

  def forward(self, input):
    x, neighbors = input
    h = torch.nn.functional.relu(self.aggr1(x, neighbors))
    h = torch.nn.functional.relu(self.aggr2(h, neighbors))
    logits = self.aggr3(h, neighbors)
    return logits

In [12]:
model = GraphSAGE()
x = data.x[0]
mask = data.edge_index[0] == 0
node_neighbors = torch.stack([data.x[i] for i in data.edge_index[1][mask]])

y = data.y[0]
model((x, node_neighbors))
# print(model.aggr2.w)

tensor([-0.0230,  0.0044, -0.0341,  0.0342,  0.0241, -0.0357, -0.1257, -0.0075,
        -0.0434, -0.0055, -0.0643,  0.0931, -0.0191,  0.0005,  0.0395, -0.0182,
        -0.0125,  0.0066, -0.0518, -0.0556,  0.0033, -0.0376, -0.0341,  0.0505,
         0.0579,  0.0084,  0.1447,  0.0144, -0.0540,  0.1284, -0.0007,  0.0128,
         0.0483, -0.0189, -0.0183,  0.0078,  0.0433,  0.0852, -0.0896, -0.0015,
         0.0092,  0.0155,  0.0103,  0.0422,  0.0793, -0.0095,  0.0934,  0.1117,
        -0.0490, -0.0435, -0.1044, -0.0177, -0.0536,  0.0043,  0.0141,  0.0512,
         0.0259, -0.0688,  0.0282, -0.0557, -0.0899,  0.0236,  0.0986,  0.0154,
        -0.0231, -0.0244,  0.0287, -0.0281, -0.0715, -0.0588],
       grad_fn=<SqueezeBackward4>)

In [14]:
model = GraphSAGE()
loss_fn= torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

epochs = 1
for epoch in range(100):
  for i in range(len(data.x[0])):
    optimizer.zero_grad()
    x = data.x[i]
    mask = data.edge_index[0] == i
    node_neighbors = torch.stack([data.x[i] for i in data.edge_index[1][mask]])

    # node_neighbors = torch.stack([data.x[v] for u, v in zip(data.edge_index[0], data.edge_index[1]) if data.edge_index[0][u] == i])
    y = data.y[i]
    y = y.unsqueeze(0)
    pred = model((x, node_neighbors))
    pred = pred.unsqueeze(0)
    loss = loss_fn(pred, y)
    loss.backward()
    optimizer.step()
    print(loss)

tensor(4.2482, grad_fn=<NllLossBackward0>)
tensor(4.2538, grad_fn=<NllLossBackward0>)
tensor(4.0695, grad_fn=<NllLossBackward0>)
tensor(4.1361, grad_fn=<NllLossBackward0>)
tensor(3.7441, grad_fn=<NllLossBackward0>)
tensor(3.5899, grad_fn=<NllLossBackward0>)
tensor(3.9139, grad_fn=<NllLossBackward0>)
tensor(3.7674, grad_fn=<NllLossBackward0>)
tensor(3.7593, grad_fn=<NllLossBackward0>)
tensor(3.7307, grad_fn=<NllLossBackward0>)
tensor(3.3861, grad_fn=<NllLossBackward0>)
tensor(3.4778, grad_fn=<NllLossBackward0>)
tensor(4.2726, grad_fn=<NllLossBackward0>)
tensor(3.1753, grad_fn=<NllLossBackward0>)
tensor(3.0753, grad_fn=<NllLossBackward0>)
tensor(3.0203, grad_fn=<NllLossBackward0>)
tensor(2.8119, grad_fn=<NllLossBackward0>)
tensor(2.5446, grad_fn=<NllLossBackward0>)
tensor(2.4031, grad_fn=<NllLossBackward0>)
tensor(4.1439, grad_fn=<NllLossBackward0>)
tensor(2.2511, grad_fn=<NllLossBackward0>)
tensor(2.6352, grad_fn=<NllLossBackward0>)
tensor(3.8180, grad_fn=<NllLossBackward0>)
tensor(1.85

KeyboardInterrupt: 